# 17. Concurrency with Futures

긴 지연 시간 동안 CPU 클록을 낭비하지 않기 위해 네트워크 입출력을 효율적으로 처리하려면 동시성을 이용해야 한다. 네트워크에서 응답이 오는 동안 다른 일을 처리하는 것이 좋다.

In [1]:
import os
import time
import sys

import requests   # 표준 라이브러리가 아니기에 관례에 따라 한 줄을 띄웠다.

POP20_CC = 'CN IN US ID BR PK NG BD RU JP MX PH VN ET EG DE IR TR CD FR'.split()
BASE_URL = 'http://flupy.org/data/flags'
DEST_DIR = 'downloads/'

def save_flag(img, filename) :
    path = os.path.join(DEST_DIR, filename)
    # with open(path, 'wb') as fp : fp.write(img)
    # 이 줄이 있으면 오류가 발생해서 어쩔 수 없이 주석 처리를 하였다.

def get_flag(cc) :
    url = '{}/{cc}/{cc}.gif'.format(BASE_URL, cc=cc.lower())
    resp = requests.get(url)
    return resp.content

def show(text) :
    print(text, end=' ')
    sys.stdout.flush()

def download_many(cc_list) :
    for cc in sorted(cc_list) :
        image = get_flag(cc)
        show(cc)
        save_flag(image, cc.lower() + '.gif')
    return len(cc_list)

def main(download_many) :
    t0 = time.time()
    count = download_many(POP20_CC)
    elapsed = time.time() - t0
    msg = '\n{} flags downloads in {:.2f}s'
    print(msg.format(count, elapsed))

if __name__ == '__main__' :
    main(download_many)

BD BR CD CN DE EG ET FR ID IN IR JP MX NG PH PK RU TR US VN 
20 flags downloads in 2.92s


이를 $\texttt{concurrent.futures}$를 이용하여 구현하면 다음과 같다.

In [2]:
from concurrent import futures

MAX_WORKERS = 20

def download_one(cc):
    image = get_flag(cc)
    show(cc)
    save_flag(image, cc.lower() + '.gif')
    return cc

def download_many2(cc_list):
    workers = min(MAX_WORKERS, len(cc_list))
    with futures.ThreadPoolExecutor(workers) as executor:
        res = executor.map(download_one, sorted(cc_list))
    
    return len(list(res))

if __name__ == '__main__':
    main(download_many2)

BR FR PH ID CNPK IN DE ET TR IR  JP RU VN NG CD BD EG MX US 
20 flags downloads in 0.26s


보다시피 다운로드에 걸리는 시간이 확연히 차이 나는 것을 볼 수 있다.

# 18. Concurrency with asyncio

아래는 장시간 연산이 실행되는 동안 threading 모듈의 스레드를 이용해서 콘솔에 '|/-\' 아스키 문자로 스피너 애니메이션을 보여주는 예시다. (현재는 콘솔 환경이 아니라서 일련의 문자열이 보인다.)

In [3]:
import threading
import itertools
import time
import sys

class Signal :
    go = True

def spin(msg, signal) :
    write, flush = sys.stdout.write, sys.stdout.flush
    for char in itertools.cycle('|/-\\') :
        status = char + ' ' + msg
        write(status)
        flush()
        write('\x08' * len(status))
        time.sleep(.1)
        if not signal.go : break
    write(' ' * len(status) + '\x08' * len(status))

def slow_function() :
    # 입출력을 위해 장시간 기다리는 것처럼 보이게 만든다.
    time.sleep(3)
    return 42

def supervisor() :
    signal = Signal()
    spinner = threading.Thread(target=spin, args=('thinking!', signal))
    print('spinner object:', spinner)
    spinner.start()
    result = slow_function()
    signal.go = False
    spinner.join()
    return result

def main() :
    result = supervisor()
    print('Answer:', result)

if __name__ == '__main__' :
    main()

spinner object: <Thread(Thread-6, initial)>
| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking          Answer: 42


이를 asyncio.coroutine을 이용해서 다시 구현한 것이 아래 코드이다.

In [8]:
import asyncio
import itertools
import sys

@asyncio.coroutine
def spin(msg) :
    write, flush = sys.stdout.write, sys.stdout.flush
    for char in itertools.cycle('|/-\\') :
        status = char + ' ' + msg
        write(status)
        flush()
        write('\x08' * len(status))
        try :
            yield from asyncio.sleep(.1)
        except asyncio.CancelledError :
            break
    write(' ' * len(status) + '\x08' * len(status))

@asyncio.coroutine
def slow_function() :
    # 입출력을 위해 장시간 기다리는 것처럼 보이게 만든다.
    yield from asyncio.sleep(3)
    return 42

@asyncio.coroutine
def supervisor() :
    spinner = asyncio.async(spin('thinking!'))
    print('spinner object:', spinner)
    result = yield from slow_function()
    spinner.cancel()
    return result

def main() :
    loop = asyncio.get_event_loop()
    result = loop.run_until_complete(supervisor())
    loop.close()
    print('Answer:', result)

if __name__ == '__main__' :
    main()


SyntaxError: invalid syntax (<ipython-input-8-4424feb41ef8>, line 27)